# MAIN FALSE SIG FUNCIONS

In [4]:
# importing the libraries

from pycbc.types import timeseries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from scipy import signal
import matplotlib.pyplot as plt
from numpy.random import uniform, randint
import pycbc.noise
import pycbc.psd
import pycbc.filter
import pylab
from pycbc.filter import sigma
from pycbc.waveform import get_td_waveform
from pycbc.psd import interpolate
from pycbc.types.timeseries import load_timeseries

PyCBC.libutils: pkg-config call failed, setting NO_PKGCONFIG=1


The following functions simulate false waveforms in a random manner.

The waveform is saved under '/Users/DRRRR/Desktop/pyCBC/false_signal.txt'

It can be loaded as pycbc time series using load_timeseries('/Users/DRRRR/Desktop/pyCBC/false_signal.txt').

The only argument required is the length of the waveform t.

The false waveforms have randomized amlitude on the scale: 1; yet random_false_sig asks for amplitude magnitude

In [2]:
def damp_sin(t):
    # simulate a damping sinusoidal time series
    nsample = 4096
    a = uniform(0.0,200.0)   # frequency factor
    b = uniform(0.0,250.0)   # damping factor
    
    x = np.linspace(0, t, nsample+1)
    y = np.sin(x*a) / (x*b+1)
    data = pd.Series(y, index=x)
    data *= uniform(0.1,10.0)
    return data

In [59]:
def exp_sin(t):
    # simulate a damping sinusoidal time series
    nsample = 4096
    a = uniform(0.0,200.0)   # frequency factor
    b = randint(1,5)   # envelop exponential function order
    
    x = np.linspace(0, t, nsample+1)
    y = np.sin(x*a) * (x**b)
    data = pd.Series(y, index=x)
    data *= uniform(0.0,10.0)
    return data

In [4]:
def mysquare(t): 
    # simulate a square wave 
    nsample = 4096
    a = randint(1,50)    # frequency factor
    b = randint(0,10)   # periodity information
    
    x = np.linspace(0, t, nsample+1)
    sig = np.sin(b * np.pi * x)
    y = signal.square(a * np.pi * x, duty=(sig + 1)/2)
    
    data = pd.Series(y, index=x)
    data *= uniform(0.0,10.0)
    return data

In [5]:
def mysawtooth(t): 
    # simulate a square wave 
    nsample = 4096
    a = randint(1,50)   # frequency factor
    
    x = np.linspace(0, t, nsample+1)
    y = signal.sawtooth(a * np.pi * x) 
    
    data = pd.Series(y, index=x)
    data *= uniform(0.0,10.0)
    return data

In [6]:
def myGaussPulse(t):
    # simulate a Gaussian pulse 
    nsample = 4096   
    fc = randint(1,500)   # widness of waveform
    power = randint(1,6)
    even = randint(0,2)
    
    x = np.linspace(-t/2.0, t/2.0, nsample+1)
    i, q, e = signal.gausspulse(x**power, fc=fc, retquad=True, retenv=True)
    if even == 0:
        data = pd.Series(i, index=x)
    else:
        data = pd.Series(q, index=x)
    data *= uniform(0.0,10.0)
    return data

In [53]:
def myChirp(t):
    # Generate a linear chirp
    nsample = 4096  
    f0 = np.random.uniform(0.0,40.0)   # f0 how fast change, the lower the faster
    f1 = np.random.uniform(0.0,20.0)   # f1 frequency information

    x = np.linspace(0, t, nsample+1)
    y = signal.chirp(x, f0=f0, f1=f1, t1=np.random.uniform(0.0,0.8), method='linear')
    data = pd.Series(y, index=x)
    data *= uniform(0.0,10.0)
    return data

In [8]:
# randomly generate one of the following 6 types of signals.
# Create a file temporary.txt in local directory
# if the txt already exist, will update the content
# total length generated will be resized to 1 second
# magitude of the false signal, 10*mag

def random_false_sig(mag):
    i = randint(0,6)
    if i==0:
        data = damp_sin(1)
    elif i==1:            
        data = exp_sin(1)
    elif i==2:            
        data = mysquare(1)
    elif i==3:            
        data = mysawtooth(1)
    elif i==4:            
        data = myGaussPulse(1)
    else:            
        data = myChirp(1)
    data *= 10**mag 
    
    # Convert into a pycbc timeseries using the following method
    f_path = 'temporary.txt'
    f= open(f_path,"w+").close()
    data.to_csv(f_path,sep=' ')
    ts = load_timeseries(f_path)
    ts.start_time = 0
    
    return ts

In [9]:
# return the time inverse of the gravitational wave template: "flip" the template

def flip_gw(hp):  
    data = pd.Series(np.flip(np.asarray(hp)), index=hp.sample_times)
    
    #save file
    f_path = 'temporary.txt'
    f= open(f_path,"w+").close()
    data.to_csv(f_path,sep=' ')
    flipped = load_timeseries(f_path)
    
    flipped.start_time = 0
    
    return flipped

In [10]:
'''

# example

hp,hc=get_td_waveform(approximant="SEOBNRv4_opt",
                             mass1=40,    
                             mass2=55,
                             delta_t=1.0/4096,
                             f_lower=20,
                             distance=100)

pylab.plot(hp.sample_times, hp, label='original') 
pylab.xlabel("Time (s)")
pylab.show()

shifted = hp.cyclic_time_shift(0.1)
pylab.plot(shifted.sample_times, shifted, label='shifted') 
pylab.xlabel("Time (s)")
pylab.show()

flipped = flip_gw(shifted)
pylab.plot(flipped.sample_times, flipped) 
pylab.xlabel("Time (s)")
pylab.show()

'''

'\n\n# example\n\nhp,hc=get_td_waveform(approximant="SEOBNRv4_opt",\n                             mass1=40,    \n                             mass2=55,\n                             delta_t=1.0/4096,\n                             f_lower=20,\n                             distance=100)\n\npylab.plot(hp.sample_times, hp, label=\'original\') \npylab.xlabel("Time (s)")\npylab.show()\n\nshifted = hp.cyclic_time_shift(0.1)\npylab.plot(shifted.sample_times, shifted, label=\'shifted\') \npylab.xlabel("Time (s)")\npylab.show()\n\nflipped = flip_gw(shifted)\npylab.plot(flipped.sample_times, flipped) \npylab.xlabel("Time (s)")\npylab.show()\n\n'

# FOR STAGE 2B, FALSE SIGNALS THAT ESPECIALLY LOOK LIKE GW

In [15]:
# allow more parameter tuning than myGaussPulse
# fc: widness of waveform
# power: randint(1,6)
# even = 0 or 1


def gaussPulse_2B(t, fc, power, mag, even):
    nsample = 4096   
    
    x = np.linspace(-t/2.0, t/2.0, nsample+1)
    i, q, e = signal.gausspulse(x**power, fc=fc, retquad=True, retenv=True)
    if even == 0:
        data = pd.Series(i, index=x)
    else:
        data = pd.Series(q, index=x)
    data *= uniform(0.0,10.0)
    
    data *= 10**mag 
    
    # Convert into a pycbc timeseries using the following method
    f_path = 'temporary.txt'
    f= open(f_path,"w+").close()
    data.to_csv(f_path,sep=' ')
    ts = load_timeseries(f_path)
    ts.start_time = 0
    
    return ts

In [106]:
'''
# test (for stage 2B)
fc = uniform(10,150)
even = randint(2)
A = gaussPulse_2B(1, fc, 1, -21, even)

pylab.plot(A.sample_times, A) 
pylab.xlabel("Time (s)")
pylab.show()
'''

'\n# test (for stage 2B)\nfc = uniform(10,150)\neven = randint(2)\nA = GaussPulse_jr(1, fc, 1, -21, even)\n\npylab.plot(A.sample_times, A) \npylab.xlabel("Time (s)")\npylab.show()\n'

In [107]:
# a = uniform(0.0,200.0)   frequency factor
# b = randint(1,5)   envelop exponential function order
    
def expsin_2B(t,a,b,mag):
    # simulate a exponential sinusoidal time series
    nsample = 4096
    
    x = np.linspace(0, t, nsample+1)
    y = np.sin(x*a) * (x**b)
    data = pd.Series(y, index=x)
    data *= uniform(0.0,10.0)
    
    data *= 10**mag
    
    # Convert into a pycbc timeseries using the following method
    f_path = 'temporary.txt'
    f= open(f_path,"w+").close()
    data.to_csv(f_path,sep=' ')
    ts = load_timeseries(f_path)
    ts.start_time = 0
    
    return ts

In [129]:
'''
# test (for stage 2B)
a = uniform(130,180)
b = randint(1,3)
B = expsin_2B(1,a,b,-21)

pylab.plot(B.sample_times, B) 
pylab.xlabel("Time (s)")
pylab.show()
'''

'\n# test (for stage 2B)\na = uniform(130,180)\nb = randint(1,3)\n\nB = expsin_jr(1,a,b,-21)\n\npylab.plot(B.sample_times, B) \npylab.xlabel("Time (s)")\npylab.show()\n'

In [130]:
#a = randint(1,50)   frequency factor

def sawtooth_2B(t,a, mag): 
    # simulate a square wave 
    nsample = 4096    
    x = np.linspace(0, t, nsample+1)
    y = signal.sawtooth(a * np.pi * x) 
    
    data = pd.Series(y, index=x)
    data *= uniform(0.0,10.0)
    data *= 10**mag
    
    # Convert into a pycbc timeseries using the following method
    f_path = 'temporary.txt'
    f= open(f_path,"w+").close()
    data.to_csv(f_path,sep=' ')
    ts = load_timeseries(f_path)
    ts.start_time = 0
    
    return ts

In [141]:
'''
# test (for stage 2B)
a = randint(50,70)
C = sawtooth_2B(1,50,-21)

pylab.plot(C.sample_times, C) 
pylab.xlabel("Time (s)")
pylab.show()
'''

'\n# test (for stage 2B)\na = randint(50,70)\nC = sawtooth_jr(1,50,-21)\n\npylab.plot(C.sample_times, C) \npylab.xlabel("Time (s)")\npylab.show()\n'

In [144]:
# f0 = np.random.uniform(0.0,40.0)    f0 how fast change, the lower the faster
# f1 = np.random.uniform(0.0,20.0)    f1 frequency information

def chirp_2B(t,f0,f1,mag):
    # Generate a linear chirp
    nsample = 4096  
    
    x = np.linspace(0, t, nsample+1)
    y = signal.chirp(x, f0=f0, f1=f1, t1=np.random.uniform(0.0,0.8), method='linear')
    data = pd.Series(y, index=x)
    data *= uniform(0.0,10.0)
    data *= 10**mag
    
    # Convert into a pycbc timeseries using the following method
    f_path = 'temporary.txt'
    f= open(f_path,"w+").close()
    data.to_csv(f_path,sep=' ')
    ts = load_timeseries(f_path)
    ts.start_time = 0
    
    return ts

In [204]:
'''
# test (for stage 2B)
f0 = np.random.uniform(10,15.0)
D = chirp_2B(1,f0,19,-21)

pylab.plot(D.sample_times, D) 
pylab.xlabel("Time (s)")
pylab.show()
'''

'\n# test (for stage 2B)\nf0 = np.random.uniform(10,15.0)\nD = chirp_jr(1,f0,19,-21)\n\npylab.plot(D.sample_times, D) \npylab.xlabel("Time (s)")\npylab.show()\n'